# Observing Blocks

## KCWI Example

In [1]:
from keckODL.target import Target, TargetList
from keckODL.kcwi import KCWIblueDetectorConfig
from keckODL.kcwi import KCWIblueConfig
from keckODL.offset import Stare, StarSkyStar
from keckODL.block import ObservingBlock, ObservingBlockList

from astropy import units as u

In [2]:
ngc1333 = Target('NGC1333', rotmode='PA', PA=22.5,
                 objecttype='science',
                 acquisition='blind')

In [3]:
kcwi_1800s = KCWIblueDetectorConfig(exptime=1800, binning='2x2')
kcwi_1800s.write('kcwi_1800s.txt')
print(kcwi_1800s)

KCWIblue 1800.0s (9)


In [4]:
med_slicer_4800 = KCWIblueConfig(slicer='medium', grating='BH3', cwave=4800)
med_slicer_4800.write('KCWI_medslicer_4800.txt')
print(med_slicer_4800)

medium BH3 4800 A


In [5]:
skyoffset = StarSkyStar(dx=30*u.arcsec, dy=30*u.arcsec)
print(skyoffset)

StarSkyStar (30 30)


In [6]:
skyoffset

Frame: SkyFrame (RAOFF DECOFF)
 dx(")| dy(")| dr(deg)|    name|guide?
------|------|--------|--------|------
  +0.0|  +0.0|    +0.0|    star|  True
 +30.0| +30.0|    +0.0|     sky| False
  +0.0|  +0.0|    +0.0|    star|  True

In [7]:
for i,offset in enumerate(skyoffset):
    print(f'Executing offset: {i}')
    offset.execute()

Executing offset: 0
'RAOFF'.write(0.0, rel2base=t)
'DECOFF'.write(0.0, rel2base=t)
Executing offset: 1
'RAOFF'.write(30.0, rel2base=t)
'DECOFF'.write(30.0, rel2base=t)
Executing offset: 2
'RAOFF'.write(0.0, rel2base=t)
'DECOFF'.write(0.0, rel2base=t)


In [8]:
OB1 = ObservingBlock(target=ngc1333, pattern=skyoffset,
                            detconfig=kcwi_1800s,
                            instconfig=med_slicer_4800,
                            repeat=3)
print(OB1)

NGC1333        |StarSkyStar (30 30)|     3 |KCWIblue 1800.0s (9)               |medium BH3 4800 A                            


In [9]:
OBlist = ObservingBlockList([OB1])
OBlist

Target         |Pattern            |repeat |DetectorConfig                     |InstrumentConfig                             
---------------|-------------------|-------|-----------------------------------|---------------------------------------------
NGC1333        |StarSkyStar (30 30)|     3 |KCWIblue 1800.0s (9)               |medium BH3 4800 A                            

In [10]:
OBlist.estimate_time()

Shutter Open Time: 16200 s (4.5 hrs)
Wall Clock Time: 16200 s (4.5 hrs)


In [11]:
cals_I_need = med_slicer_4800.cals()
cals_I_need

Target         |Pattern            |repeat |DetectorConfig                     |InstrumentConfig                             
---------------|-------------------|-------|-----------------------------------|---------------------------------------------
None           |Stare              |     1 |KCWIblue 6.0s (9)                  |medium BH3 4800 A arclamp=CONT calobj=MedBarsA
None           |Stare              |     1 |KCWIblue 30.0s (9)                 |medium BH3 4800 A arclamp=FEAR calobj=FlatA  
None           |Stare              |     1 |KCWIblue 45.0s (9)                 |medium BH3 4800 A arclamp=THAR calobj=FlatA  
None           |Stare              |     6 |KCWIblue 6.0s (9)                  |medium BH3 4800 A arclamp=CONT calobj=FlatA  
None           |Stare              |     7 |KCWIblue 0.0s (9, Dark)            |medium BH3 4800 A                            
DomeFlats      |Stare              |     3 |KCWIblue 100.0s (9)                |medium BH3 4800 A domeflatlamp=True  

In [12]:
cals_I_need.estimate_time()

Shutter Open Time: 417 s (0.1 hrs)
Wall Clock Time: 417 s (0.1 hrs)


# KCWI + KCRM Example

## MOSFIRE Example

In [13]:
from keckODL.target import Target, TargetList
from keckODL.mosfire import MOSFIREConfig, MOSFIREDetectorConfig, long2pos, ABBA
from keckODL.offset import Stare
from keckODL.block import ObservingBlock, ObservingBlockList

In [14]:
v1647Ori = Target('V1647 Ori', rotmode='PA', PA=22.5,
                  objecttype='science',
                  acquisition='blind')

In [15]:
# detector configs
mosfire_180s = MOSFIREDetectorConfig(exptime=180, readoutmode='MCDS16', coadds=1)
mosfire_180s.write('mosfire_180s.txt')
mosfire_120s = MOSFIREDetectorConfig(exptime=120, readoutmode='MCDS16', coadds=1)
mosfire_120s.write('mosfire_120s.txt')
print(mosfire_180s)
print(mosfire_120s)

MOSFIRE 180.0s (MCDS16, 1 coadds)
MOSFIRE 120.0s (MCDS16, 1 coadds)


In [16]:
# instrument configs
mosfire_Y = MOSFIREConfig(mode='spectroscopy', filter='Y', mask='my_MOS_mask')
mosfire_J = MOSFIREConfig(mode='spectroscopy', filter='J', mask='my_MOS_mask')
mosfire_H = MOSFIREConfig(mode='spectroscopy', filter='H', mask='my_MOS_mask')
mosfire_K = MOSFIREConfig(mode='spectroscopy', filter='K', mask='my_MOS_mask')
print(mosfire_Y)
print(mosfire_J)
print(mosfire_H)
print(mosfire_K)

my_MOS_mask Y-spectroscopy
my_MOS_mask J-spectroscopy
my_MOS_mask H-spectroscopy
my_MOS_mask K-spectroscopy


In [17]:
abba = ABBA(offset=1.5)
print(abba)

ABBA (1.5)


/Users/joshw/git/KeckODL/keckODL/offset.py:160: OffsetWarning: No offset unit given for dy, assuming arcseconds
  category=OffsetWarning)


In [18]:
abba

Frame: MOSFIRE Slit (INSTXOFF INSTYOFF)
 dx(")| dy(")| dr(deg)|    name|guide?
------|------|--------|--------|------
  +0.0|  +1.5|    +0.0|       A|  True
  +0.0|  -1.5|    +0.0|       B|  True
  +0.0|  -1.5|    +0.0|       B|  True
  +0.0|  +1.5|    +0.0|       A|  True

In [19]:
for i,offset in enumerate(abba):
    print(f'Executing offset: {i}')
    offset.execute()

Executing offset: 0
'INSTXOFF'.write(0.0, rel2base=t)
'INSTYOFF'.write(1.5, rel2base=t)
Executing offset: 1
'INSTXOFF'.write(0.0, rel2base=t)
'INSTYOFF'.write(-1.5, rel2base=t)
Executing offset: 2
'INSTXOFF'.write(0.0, rel2base=t)
'INSTYOFF'.write(-1.5, rel2base=t)
Executing offset: 3
'INSTXOFF'.write(0.0, rel2base=t)
'INSTYOFF'.write(1.5, rel2base=t)


In [20]:
myOBs = ObservingBlockList([ObservingBlock(target=v1647Ori, pattern=abba, detconfig=mosfire_180s, instconfig=mosfire_Y, repeat=5),
                            ObservingBlock(target=v1647Ori, pattern=abba, detconfig=mosfire_120s, instconfig=mosfire_J, repeat=8),
                            ObservingBlock(target=v1647Ori, pattern=abba, detconfig=mosfire_120s, instconfig=mosfire_H, repeat=8),
                            ObservingBlock(target=v1647Ori, pattern=abba, detconfig=mosfire_180s, instconfig=mosfire_K, repeat=5),
                           ])
print(myOBs)

Target         |Pattern            |repeat |DetectorConfig                     |InstrumentConfig                             
---------------|-------------------|-------|-----------------------------------|---------------------------------------------
V1647 Ori      |ABBA (1.5)         |     5 |MOSFIRE 180.0s (MCDS16, 1 coadds)  |my_MOS_mask Y-spectroscopy                   
V1647 Ori      |ABBA (1.5)         |     8 |MOSFIRE 120.0s (MCDS16, 1 coadds)  |my_MOS_mask J-spectroscopy                   
V1647 Ori      |ABBA (1.5)         |     8 |MOSFIRE 120.0s (MCDS16, 1 coadds)  |my_MOS_mask H-spectroscopy                   
V1647 Ori      |ABBA (1.5)         |     5 |MOSFIRE 180.0s (MCDS16, 1 coadds)  |my_MOS_mask K-spectroscopy                   


In [21]:
myOBs.estimate_time()

Shutter Open Time: 14880 s (4.1 hrs)
Wall Clock Time: 14880 s (4.1 hrs)


In [22]:
cals_I_need = mosfire_Y.cals()
cals_I_need.extend( mosfire_J.cals() )
cals_I_need.extend( mosfire_H.cals() )
cals_I_need.extend( mosfire_K.cals() )
print(cals_I_need)

Target         |Pattern            |repeat |DetectorConfig                     |InstrumentConfig                             
---------------|-------------------|-------|-----------------------------------|---------------------------------------------
DomeFlats      |Stare              |     7 |MOSFIRE 11.0s (CDS, 1 coadds)      |my_MOS_mask Y-spectroscopy domeflatlamp=True 
DomeFlats      |Stare              |     7 |MOSFIRE 11.0s (CDS, 1 coadds)      |my_MOS_mask J-spectroscopy domeflatlamp=True 
DomeFlats      |Stare              |     7 |MOSFIRE 11.0s (CDS, 1 coadds)      |my_MOS_mask H-spectroscopy domeflatlamp=True 
DomeFlats      |Stare              |     7 |MOSFIRE 11.0s (CDS, 1 coadds)      |my_MOS_mask K-spectroscopy domeflatlamp=True 
DomeFlats      |Stare              |     7 |MOSFIRE 11.0s (CDS, 1 coadds)      |my_MOS_mask K-spectroscopy domeflatlamp=False
None           |Stare              |     2 |MOSFIRE 1.0s (CDS, 1 coadds)       |my_MOS_mask K-spectroscopy arclamp=Ne 

In [23]:
cals_I_need.estimate_time()

Shutter Open Time: 389 s (0.1 hrs)
Wall Clock Time: 389 s (0.1 hrs)


## MOSFIRE Example 2

In [24]:
singleobj_Y = MOSFIREConfig(mode='spectroscopy', filter='Y', mask='longslit_3x0.7')
singleobj_J = MOSFIREConfig(mode='spectroscopy', filter='J', mask='longslit_3x0.7')
singleobj_H = MOSFIREConfig(mode='spectroscopy', filter='H', mask='long2pos')
singleobj_K = MOSFIREConfig(mode='spectroscopy', filter='K', mask='long2pos')
print(singleobj_Y)
print(singleobj_J)
print(singleobj_H)
print(singleobj_K)

longslit_3x0.7 Y-spectroscopy
longslit_3x0.7 J-spectroscopy
long2pos H-spectroscopy
long2pos K-spectroscopy


In [25]:
long2pos_pattern = long2pos()
print(long2pos_pattern)

long2pos


/Users/joshw/git/KeckODL/keckODL/offset.py:155: OffsetWarning: No offset unit given for dx, assuming arcseconds
  category=OffsetWarning)


In [26]:
long2pos_pattern

Frame: MOSFIRE Detector (INSTXOFF INSTYOFF)
 dx(")| dy(")| dr(deg)|    name|guide?
------|------|--------|--------|------
 +45.0| -23.0|    +0.0|       A|  True
 +45.0|  -9.0|    +0.0|       B|  True
 -45.0|  +9.0|    +0.0|       A|  True
 -45.0| +23.0|    +0.0|       B|  True

In [27]:
long2pos_pattern[0].execute()

'INSTXOFF'.write(45.0, rel2base=t)
'INSTYOFF'.write(-23.0, rel2base=t)


In [28]:
myOBs2 = ObservingBlockList([ObservingBlock(target=v1647Ori, pattern=abba, detconfig=mosfire_180s, instconfig=singleobj_Y, repeat=2),
                             ObservingBlock(target=v1647Ori, pattern=abba, detconfig=mosfire_120s, instconfig=singleobj_J, repeat=2),
                             ObservingBlock(target=v1647Ori, pattern=long2pos_pattern, detconfig=mosfire_120s, instconfig=singleobj_H, repeat=2),
                             ObservingBlock(target=v1647Ori, pattern=long2pos_pattern, detconfig=mosfire_180s, instconfig=singleobj_K, repeat=2),
                            ])
print(myOBs2)

Target         |Pattern            |repeat |DetectorConfig                     |InstrumentConfig                             
---------------|-------------------|-------|-----------------------------------|---------------------------------------------
V1647 Ori      |ABBA (1.5)         |     2 |MOSFIRE 180.0s (MCDS16, 1 coadds)  |longslit_3x0.7 Y-spectroscopy                
V1647 Ori      |ABBA (1.5)         |     2 |MOSFIRE 120.0s (MCDS16, 1 coadds)  |longslit_3x0.7 J-spectroscopy                
V1647 Ori      |long2pos           |     2 |MOSFIRE 120.0s (MCDS16, 1 coadds)  |long2pos H-spectroscopy                      
V1647 Ori      |long2pos           |     2 |MOSFIRE 180.0s (MCDS16, 1 coadds)  |long2pos K-spectroscopy                      


In [29]:
myOBs2.estimate_time()

Shutter Open Time: 4800 s (1.3 hrs)
Wall Clock Time: 4800 s (1.3 hrs)


In [30]:
cals_I_need = singleobj_Y.cals()
cals_I_need.extend(singleobj_J.cals())
cals_I_need.extend(singleobj_H.cals())
cals_I_need.extend(singleobj_K.cals())
print(cals_I_need)

Target         |Pattern            |repeat |DetectorConfig                     |InstrumentConfig                             
---------------|-------------------|-------|-----------------------------------|---------------------------------------------
DomeFlats      |Stare              |     7 |MOSFIRE 11.0s (CDS, 1 coadds)      |longslit_3x0.7 Y-spectroscopy domeflatlamp=True
DomeFlats      |Stare              |     7 |MOSFIRE 11.0s (CDS, 1 coadds)      |longslit_3x0.7 J-spectroscopy domeflatlamp=True
DomeFlats      |Stare              |     7 |MOSFIRE 11.0s (CDS, 1 coadds)      |long2pos H-spectroscopy domeflatlamp=True    
DomeFlats      |Stare              |     7 |MOSFIRE 11.0s (CDS, 1 coadds)      |long2pos K-spectroscopy domeflatlamp=True    
DomeFlats      |Stare              |     7 |MOSFIRE 11.0s (CDS, 1 coadds)      |long2pos K-spectroscopy domeflatlamp=False   
None           |Stare              |     2 |MOSFIRE 1.0s (CDS, 1 coadds)       |long2pos K-spectroscopy arclamp=Ne

In [31]:
cals_I_need.estimate_time()

Shutter Open Time: 389 s (0.1 hrs)
Wall Clock Time: 389 s (0.1 hrs)


# MOSFIRE Night Example

In [32]:
myOBs3 = ObservingBlockList([ObservingBlock(target=ngc1333, pattern=abba, detconfig=mosfire_180s, instconfig=singleobj_Y, repeat=2),
                             ObservingBlock(target=ngc1333, pattern=abba, detconfig=mosfire_120s, instconfig=singleobj_J, repeat=2),
                             ObservingBlock(target=ngc1333, pattern=long2pos_pattern, detconfig=mosfire_120s, instconfig=singleobj_H, repeat=2),
                             ObservingBlock(target=ngc1333, pattern=long2pos_pattern, detconfig=mosfire_180s, instconfig=singleobj_K, repeat=2),
                            ])
print(myOBs3)

Target         |Pattern            |repeat |DetectorConfig                     |InstrumentConfig                             
---------------|-------------------|-------|-----------------------------------|---------------------------------------------
NGC1333        |ABBA (1.5)         |     2 |MOSFIRE 180.0s (MCDS16, 1 coadds)  |longslit_3x0.7 Y-spectroscopy                
NGC1333        |ABBA (1.5)         |     2 |MOSFIRE 120.0s (MCDS16, 1 coadds)  |longslit_3x0.7 J-spectroscopy                
NGC1333        |long2pos           |     2 |MOSFIRE 120.0s (MCDS16, 1 coadds)  |long2pos H-spectroscopy                      
NGC1333        |long2pos           |     2 |MOSFIRE 180.0s (MCDS16, 1 coadds)  |long2pos K-spectroscopy                      


In [33]:
myOBs.extend(myOBs2)
myOBs.extend(myOBs3)

In [34]:
myOBs

Target         |Pattern            |repeat |DetectorConfig                     |InstrumentConfig                             
---------------|-------------------|-------|-----------------------------------|---------------------------------------------
V1647 Ori      |ABBA (1.5)         |     5 |MOSFIRE 180.0s (MCDS16, 1 coadds)  |my_MOS_mask Y-spectroscopy                   
V1647 Ori      |ABBA (1.5)         |     8 |MOSFIRE 120.0s (MCDS16, 1 coadds)  |my_MOS_mask J-spectroscopy                   
V1647 Ori      |ABBA (1.5)         |     8 |MOSFIRE 120.0s (MCDS16, 1 coadds)  |my_MOS_mask H-spectroscopy                   
V1647 Ori      |ABBA (1.5)         |     5 |MOSFIRE 180.0s (MCDS16, 1 coadds)  |my_MOS_mask K-spectroscopy                   
V1647 Ori      |ABBA (1.5)         |     2 |MOSFIRE 180.0s (MCDS16, 1 coadds)  |longslit_3x0.7 Y-spectroscopy                
V1647 Ori      |ABBA (1.5)         |     2 |MOSFIRE 120.0s (MCDS16, 1 coadds)  |longslit_3x0.7 J-spectroscopy         

In [35]:
myOBs.cals()

Target         |Pattern            |repeat |DetectorConfig                     |InstrumentConfig                             
---------------|-------------------|-------|-----------------------------------|---------------------------------------------
DomeFlats      |Stare              |     7 |MOSFIRE 11.0s (CDS, 1 coadds)      |my_MOS_mask K-spectroscopy domeflatlamp=True 
DomeFlats      |Stare              |     7 |MOSFIRE 11.0s (CDS, 1 coadds)      |my_MOS_mask K-spectroscopy domeflatlamp=False
None           |Stare              |     2 |MOSFIRE 1.0s (CDS, 1 coadds)       |my_MOS_mask K-spectroscopy arclamp=Ne        
None           |Stare              |     2 |MOSFIRE 1.0s (CDS, 1 coadds)       |my_MOS_mask K-spectroscopy arclamp=Ar        
DomeFlats      |Stare              |     7 |MOSFIRE 11.0s (CDS, 1 coadds)      |my_MOS_mask J-spectroscopy domeflatlamp=True 
DomeFlats      |Stare              |     7 |MOSFIRE 11.0s (CDS, 1 coadds)      |long2pos H-spectroscopy domeflatlamp=T